In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('..')

from backtest.performance_generater import PerformanceGenerator
from backtest.operators import *

In [8]:
open = pd.read_csv('/Volumes/Database/Factor_data/open.csv', index_col=0)
close = pd.read_csv('/Volumes/Database/Factor_data/close.csv', index_col=0)
returns = pd.read_csv('/Volumes/Database/Factor_data/return.csv', index_col=0)

# 要再確認一下 報酬正確性
unadj_return = close.pct_change().shift(-2) 
exp_returns = returns.shift(-2) * 0.01

# operatingmargin = pd.read_csv('/Volumes/Database/Factor_data/營業利益率_90.csv', index_col=0) * 0.01
# pb_q = pd.read_csv('/Volumes/Database/Factor_data/當季季底PB_90.csv', index_col=0)
# revenue_m = pd.read_csv('/Volumes/Database/Factor_data/以合併為主單月營收(千元)_90.csv', index_col=0)
# operatingincome = pd.read_csv('/Volumes/Database/Factor_data/營業利益_90.csv', index_col=0)
# operatingincome = pd.read_csv('/Volumes/Database/Factor_data/單月營業利益(千元)_90.csv', index_col=0)
# outstanding = pd.read_csv('/Volumes/Database/Factor_data/流通在外股數(千股).csv', index_col=0)
# CR = pd.read_csv('/Volumes/Database/Factor_data/流動比率_90.csv', index_col=0)
CD = pd.read_csv('/Volumes/Database/Factor_data/流動負債_90.csv', index_col=0)
# CA = pd.read_csv('/Volumes/Database/Factor_data/流動資產_90.csv', index_col=0)
# inv = pd.read_csv('/Volumes/Database/Factor_data/  存貨_90.csv', index_col=0)
asset = pd.read_csv('/Volumes/Database/Factor_data/資產總額_90.csv', index_col=0)
# netasset = pd.read_csv('/Volumes/Database/Factor_data/淨值資產_90.csv', index_col=0)
# netdebt = pd.read_csv('/Volumes/Database/Factor_data/淨負債_90.csv', index_col=0)
# NI = pd.read_csv('/Volumes/Database/Factor_data/稀釋稅後淨利_90.csv', index_col=0)
# revenue_yoy = pd.read_csv('/Volumes/Database/Factor_data/單月營收成長率％_90.csv', index_col=0)
# DR = pd.read_csv('/Volumes/Database/Factor_data/負債比率_90.csv', index_col=0)
totalD = pd.read_csv('/Volumes/Database/Factor_data/負債總額_90.csv', index_col=0)


/var/folders/fx/jn0gghjd4cg68lr15nlpsbqr0000gn/T/ipykernel_3597/728278506.py:6: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [3]:
ZTXA_close = pd.read_csv('/Volumes/Database/Factor_data/ZXTA_close.csv', index_col=0)
ZTXA_close.index = pd.to_datetime(ZTXA_close.index)
market_returns = ZTXA_close['ZTXA'].pct_change()
Benchmark = ZTXA_close['ZTXA'].pct_change().shift(-2)

In [4]:
# factor = rolling_regression(operatingincome/asset, returns, 315, 'alpha')

In [11]:
CD = pd.read_csv('/Volumes/Database/Factor_data/流動負債_90.csv', index_col=0)
totalD = pd.read_csv('/Volumes/Database/Factor_data/負債總額_90.csv', index_col=0)
asset = pd.read_csv('/Volumes/Database/Factor_data/資產總額_90.csv', index_col=0)
factor = ts_pct_change((totalD - CD)/ asset, 252)

# no fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     buy_fee = 0, 
                                                     sell_fee = 0,
                                                     benchmark = Benchmark
                                                     ).backtest()

# fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     benchmark = Benchmark
                                                     ).backtest()

+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   | Turnover |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
| Performance |       253.14 %        |       134.58 %       |     1.03     |    12.96 %     |   29.72 %    |  12.66 %   | 0.80 % |  0.29 %  |
|  Benchmark  |       145.24 %        |       103.31 %       |     0.62     |     9.05 %     |   31.51 %    |  16.17 %   | 1.02 % |   nan    |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+


+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   | Turnover |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
| Performance |       243.39 %        |       131.77 %       |     1.00     |    12.65 %     |   29.72 %    |  12.66 %   | 0.80 % |  0.29 %  |
|  Benchmark  |       145.24 %        |       103.31 %       |     0.62     |     9.05 %     |   31.51 %    |  16.17 %   | 1.02 % |   nan    |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+


In [12]:
csv_file = '/Volumes/Database/medina_alpha/longtermdebtratio.csv'
factor.to_csv(csv_file)